In [1]:
import sqlite3
import pandas as pd
from IPython.core.display import HTML, Markdown

In [2]:
class Report:
    def __init__(self):
        self.md = ""

    def addTitle(self, title):
        self.md += f"# {title}\n\n"

    def addSubTitle(self, title):
        self.md += f"## {title}\n\n"

    def addParagraph(self, text):
        for line in text.splitlines():
            self.md += f"{line}\n"
        self.md += "\n"

    def addText(self, text):
        self.addParagraph(text)
    
    def append(self, text):
        self.md += text + "\n"

    def clear(self):
        self.md = ""

    def debug(self):
        print(self.md)

    def display(self):
        display(Markdown(self.md))

    def markdown(self):
        return self.md

    def saveToFile(self, fn):
        with open(fn, 'w') as f:
            f.write(self.md)

    def addLessonsTab(self, e, con):
        sql = f"""SELECT
                    giorno,
                    L.data as dataISO,
                    substr(L.data, 9, 2) || '/' || substr(L.data, 6, 2) || '/' || substr(L.data, 1, 4) as data,
                    L.idFasciaOraria,
                    L.idLezione
                FROM
                    Edizione E INNER JOIN 
                    Lezione L ON E.idEdizione=L.idEdizione INNER JOIN
                    Giorno G ON cast (strftime('%w', L.data) as integer)=G.idGiorno
                WHERE L.idEdizione = '{e}'
                ORDER BY E.idEdizione, L.data, L.idFasciaOraria"""
        ls = pd.read_sql_query(sql, con)
        str = "| Incontro | Data | Ora | Incontro | Data | Ora |\n|--:|:-:|:-:|--:|:-:|:-:|\n"
        for j in range(len(ls)):
            l = ls.iloc[j]
            str += f"|{l["idLezione"]}|{l["giorno"].lower()} {l["data"]} |{l["idFasciaOraria"]}"
            if j % 2 == 0:
                str += ""
            else:
                str += "|\n"
        self.md += str + "\n\n"

    def addLessonsList(self, e, con):
        sql = f"""SELECT
                    giorno,
                    L.data as dataISO,
                    substr(L.data, 9, 2) || '/' || substr(L.data, 6, 2) || '/' || substr(L.data, 1, 4) as data,
                    L.idFasciaOraria,
                    L.idLezione
                FROM
                    Edizione E INNER JOIN 
                    Lezione L ON E.idEdizione=L.idEdizione INNER JOIN
                    Giorno G ON cast (strftime('%w', L.data) as integer)=G.idGiorno
                WHERE L.idEdizione = '{e}'
                ORDER BY E.idEdizione, L.data, L.idFasciaOraria"""
        ls = pd.read_sql_query(sql, con)
        str = "\n"
        for j in range(len(ls)):
            l = ls.iloc[j]
            str += f"{l["idLezione"]}. {l["giorno"].lower()} {l["data"]} {l["idFasciaOraria"]}<sup>a</sup> ora\n"
        self.md += str + "\n\n"
        
    def addLessonsByStaff(self, p, con):
        sql = f"""SELECT
                rank() OVER (ORDER BY L.data, L.idFasciaOraria) AS progressivo,
            	E.idClasse,
            	giorno,
            	L.data AS dataISO,
            	L.idFasciaOraria,
            	E.idCorso ,
            	L.idLezione,
            	F.ruolo,
            	substr(L.data, 9, 2) || '/' || substr(L.data, 6, 2) || '/' || substr(L.data, 1, 4) as data
            FROM
            	Formatore F INNER JOIN
            	Personale P ON F.idPersonale=P.idPersonale INNER JOIN
            	Edizione E ON F.idEdizione=E.idEdizione INNER JOIN
            	Lezione L ON E.idEdizione=L.idEdizione INNER JOIN
            	Giorno G ON cast (strftime('%w', L.data) as integer)=G.idGiorno
            WHERE
            	P.idPersonale = '{p}'
                AND L.data > date()
            ORDER BY
            	L.data,
            	L.idFasciaOraria"""
        ls = pd.read_sql_query(sql, con)
        str = "|Progr.| Data | Ora | Corso | Lezione | Classe |\n|--:|:-:|:-:|:-:|--:|:-:|\n"
        for j in range(len(ls)):
            l = ls.iloc[j]
            str += f"|{l["progressivo"]}|{l["giorno"].lower()} {l["data"]}|{l["idFasciaOraria"]}|{l["idCorso"]}|{l["idLezione"]}|{l["idClasse"]}|\n"    
        self.md += str + "\n\n"

    def addDailySchedule(self, p, con):
        sql = f"""SELECT
				rank() OVER (ORDER BY L.data) AS progressivo,
            	giorno,
            	substr(L.data, 9, 2) || '/' || substr(L.data, 6, 2) || '/' || substr(L.data, 1, 4) AS data,
				group_concat(L.idFasciaOraria ORDER BY L.idFasciaOraria ASC) AS ore,
				group_concat(DISTINCT E.idCorso) AS corsi
            FROM
            	Formatore F INNER JOIN
            	Personale P ON F.idPersonale=P.idPersonale INNER JOIN
            	Edizione E ON F.idEdizione=E.idEdizione INNER JOIN
            	Lezione L ON E.idEdizione=L.idEdizione INNER JOIN
            	Giorno G ON cast (strftime('%w', L.data) as integer)=G.idGiorno
            WHERE
            	P.idPersonale = '{p}'
                AND L.data > date()
			GROUP BY L.data
            ORDER BY
            	L.data"""
        str = "|Giornata| Data | Ore | Corsi |\n|--:|:-:|:-:|:-:|\n"
        ds = pd.read_sql_query(sql, con)
        for j in range(len(ds)):
            d = ds.iloc[j]
            str += f"|{d["progressivo"]}|{d["giorno"].lower()} {d["data"]}|{d["ore"]}|{d["corsi"]}|\n"
        self.md += str + "\n\n"

    def addFullCalendar(self, con):
        sql = f"""SELECT
                  giorno
                , substr(L.data, 9, 2) || '/' || substr(L.data, 6, 2) || '/' || substr(L.data, 1, 4) AS data
                , L.idFasciaOraria
                , group_concat(F.idPersonale) AS personale
                , E.idEdizione
                , E.idClasse
                , RANK () OVER (PARTITION BY E.idEdizione ORDER BY L.data, L.idFasciaOraria) AS progressivoOre
            FROM 
                Formatore F INNER JOIN
                Personale P ON F.idPersonale=P.idPersonale INNER JOIN
                Edizione E ON F.idEdizione=E.idEdizione INNER JOIN
                Lezione L ON E.idEdizione=L.idEdizione INNER JOIN
                Giorno G ON cast (strftime('%w', L.data) as integer)=G.idGiorno
            GROUP BY L.data, L.idFasciaOraria, E.idEdizione
            ORDER BY L.data, L.idFasciaOraria, E.idEdizione"""
        str = "|Giorno | Ora | Corso | Persone | Classe | Progr. ore |\n|--:|:-:|:-:|:-:|:-:|:-:|\n"
        ds = pd.read_sql_query(sql, con)
        for j in range(len(ds)):
            d = ds.iloc[j]
            str += f"|{d["giorno"].lower()} {d["data"]}|{d["idFasciaOraria"]}|{d["idEdizione"]}|{d["personale"]}|{d["idClasse"]}|{d["progressivoOre"]}|\n"
        self.md += str + "\n\n"

In [3]:
con = sqlite3.connect("data/calendario_corsi.sqlite")
report = Report()

In [4]:
report.clear()
report.addTitle("Calendario corsi")
for c in ['RelAx', 'CyberSec', 'CryptoWithBlocks', 'LabMat2', 'PhyComp', 'RicOp']: # , 'EduRobot', 'PassInf']:
    sql = "SELECT idEdizione, denominazione, descrizione, idClasse FROM Edizione INNER JOIN Corso USING (idCorso) WHERE Corso.idCorso='" + c + "' AND stato NOT IN ('CHIUSO', 'DA CHIUDERE') ORDER BY denominazione"
    edizioni = pd.read_sql_query(sql, con)
    for i in range(len(edizioni)):
        edizione = edizioni.iloc[i]
        # Formatore
        sql = f"SELECT idPersonale FROM Formatore WHERE idEdizione = '{edizione["idEdizione"]}' AND ruolo = 'Formatore'"
        formatore = pd.read_sql_query(sql, con)["idPersonale"][0]
        # Tutor
        sql = f"SELECT idPersonale FROM Formatore WHERE idEdizione = '{edizione["idEdizione"]}' AND ruolo = 'Tutor'"
        tutor = pd.read_sql_query(sql, con)["idPersonale"][0]

        sql = f"""SELECT 
                	lower(giorno) ||
                	' ' || substr(L.data, 9, 2) ||
                	'/' || substr(L.data, 6, 2) ||
                	'/' || substr(L.data, 1, 4) AS avvio
                FROM
                	Lezione L INNER JOIN 
                	Giorno G ON cast (strftime('%w', L.data) as integer)=G.idGiorno
                WHERE
                	idEdizione = '{edizione["idEdizione"]}' AND idLezione = 1"""
        dataAvvio = pd.read_sql_query(sql, con)["avvio"][0]
        sql = f"""SELECT elenco FROM ObiettiviFormativi WHERE idCorso='{c}'"""
        obiettivi = pd.read_sql_query(sql, con)["elenco"][0]
        
        editionRep = Report()

        # Titolo
        editionRep.addSubTitle(edizione["denominazione"]) # edizione["idEdizione"] + " - " + 
        
        ## Destinatari
        editionRep.addText(f"""Agli studenti della classe {edizione["idClasse"]}<br />
Alle loro famiglie<br />
Ai loro insegnanti<br />
Al formatore {formatore}<br />
Al tutor {tutor}<br />
Alla segreteria didattica<br />
All\'Ufficio personale<br />
Ai collaboratori scolastici<br />
Al DSGA M. Zagaglia<br />""")
        
        editionRep.addText(f'**Oggetto: PTOF 2024-25 Avvio corso DM65/23 Linea A edizione "{edizione["denominazione"]}"**')

        editionRep.addText(f"Si comunica che, a partire dal giorno {dataAvvio}, prenderanno avvio le attività del corso “**{edizione["denominazione"]}**”. Il percorso rientra nell’ambito del progetto scolastico “Citizen scientists of the future” M4C1I3.1-2023-1143-P-31174, C.U.P. H34D23002330006 finanziato dal Piano nazionale di ripresa e resilienza, Missione 4 – Istruzione e ricerca – Componente 1 – Potenziamento dell’offerta dei servizi di istruzione: dagli asili nido alle università – Investimento 3.1 “Nuove competenze e nuovi linguaggi”, finanziato dall’Unione europea – Next Generation EU – “Azioni di potenziamento delle competenze STEM e multilinguistiche”.")

        # Lezioni
        editionRep.addText(f"Le lezioni saranno svolte, salvo imprevisti, nelle date seguenti: ")
        # editionRep.addLessonsTab(edizione["idEdizione"], con)
        editionRep.addLessonsList(edizione["idEdizione"], con)
        # Studenti
        #editionRep.addText(f"**Studenti/Classe**: {edizione["idClasse"]}")
        # Formatore
        #editionRep.addText(f"**Formatore**: *{formatore}*")
        # Tutor
        #editionRep.addText(f"**Tutor**: *{tutor}*")
        # Descrizione
        #editionRep.addParagraph(f"**Descrizione**: {edizione["descrizione"]}")
        editionRep.addParagraph(edizione["descrizione"])
        editionRep.addParagraph("""Gli obiettivi generali del corso includono:""")
        editionRep.addParagraph(obiettivi)
        
        
        editionRep.addText("Docente referente del progetto: Prof. Gionata Massi.\n\nCordiali saluti.\n\nAncona, xx.01.2025")
        editionRep.saveToFile("./gen_docs/editions/" + edizione["idEdizione"] + ".md")
        report.append(editionRep.markdown())
report.display()

# Calendario corsi



In [5]:
report.clear()
# Orario formatori
report.addTitle("Orario del personale")
for p in ['Lorenzoni', 'Massi', 'Pomili', 'Sciamanna', 'Taccari']:
    staffLessonsRep = Report()
    staffLessonsRep.addSubTitle(f"{p} - Lezioni")
    staffLessonsRep.addLessonsByStaff(p, con)
    staffLessonsRep.saveToFile("./gen_docs/lessons/" + p.lower() + ".md")
    report.append(staffLessonsRep.markdown())

report.display()

# Orario del personale

## Lorenzoni - Lezioni

|Progr.| Data | Ora | Corso | Lezione | Classe |
|--:|:-:|:-:|:-:|--:|:-:|



## Massi - Lezioni

|Progr.| Data | Ora | Corso | Lezione | Classe |
|--:|:-:|:-:|:-:|--:|:-:|



## Pomili - Lezioni

|Progr.| Data | Ora | Corso | Lezione | Classe |
|--:|:-:|:-:|:-:|--:|:-:|



## Sciamanna - Lezioni

|Progr.| Data | Ora | Corso | Lezione | Classe |
|--:|:-:|:-:|:-:|--:|:-:|



## Taccari - Lezioni

|Progr.| Data | Ora | Corso | Lezione | Classe |
|--:|:-:|:-:|:-:|--:|:-:|





In [6]:
report.clear()
# Impegni giornalieri
report.addTitle("Orario del personale")
for p in ['Lorenzoni', 'Massi', 'Pomili', 'Sciamanna', 'Taccari']:
    schedRep = Report()
    schedRep.addSubTitle(f"{p} - Giorni")
    schedRep.addDailySchedule(p, con)
    schedRep.saveToFile("./gen_docs/schedule/" + p.lower() + ".md")
    report.append(schedRep.markdown())

report.display()

# Orario del personale

## Lorenzoni - Giorni

|Giornata| Data | Ore | Corsi |
|--:|:-:|:-:|:-:|



## Massi - Giorni

|Giornata| Data | Ore | Corsi |
|--:|:-:|:-:|:-:|



## Pomili - Giorni

|Giornata| Data | Ore | Corsi |
|--:|:-:|:-:|:-:|



## Sciamanna - Giorni

|Giornata| Data | Ore | Corsi |
|--:|:-:|:-:|:-:|



## Taccari - Giorni

|Giornata| Data | Ore | Corsi |
|--:|:-:|:-:|:-:|





In [7]:
report.clear()
# Impegni giornalieri
report.addTitle("Calendario")
schedRep = Report()
schedRep.addSubTitle(f"Lezioni giornaliere")
schedRep.addFullCalendar(con)
schedRep.saveToFile("./gen_docs/calendar.md")
report.append(schedRep.markdown())

report.display()

# Calendario

## Lezioni giornaliere

|Giorno | Ora | Corso | Persone | Classe | Progr. ore |
|--:|:-:|:-:|:-:|:-:|:-:|
|mer 11/12/2024|3|RelAx5E|Massi,Sciamanna|S5E|1|
|mer 11/12/2024|4|RelAx4F|Massi,Sciamanna|S4F|1|
|ven 13/12/2024|5|RelAx5LSinf|Massi,Pomili|S5B+S5D|1|
|lun 16/12/2024|1|RelAx4F|Massi,Sciamanna|S4F|2|
|lun 16/12/2024|2|RelAx5E|Massi,Sciamanna|S5E|2|
|mer 18/12/2024|3|RelAx5E|Massi,Sciamanna|S5E|3|
|mer 18/12/2024|4|RelAx4F|Massi,Sciamanna|S4F|3|
|ven 20/12/2024|5|RelAx5LSinf|Massi,Pomili|S5B+S5D|2|
|mer 08/01/2025|3|RelAx5E|Massi,Sciamanna|S5E|4|
|mer 08/01/2025|4|RelAx4F|Massi,Sciamanna|S4F|4|
|ven 10/01/2025|5|RelAx5LSinf|Massi,Pomili|S5B+S5D|3|
|lun 13/01/2025|1|RelAx4F|Massi,Sciamanna|S4F|5|
|lun 13/01/2025|2|RelAx5E|Massi,Sciamanna|S5E|5|
|mer 15/01/2025|3|RelAx5E|Massi,Sciamanna|S5E|6|
|mer 15/01/2025|4|RelAx4F|Massi,Sciamanna|S4F|6|
|ven 17/01/2025|5|RelAx5LSinf|Massi,Pomili|S5B+S5D|4|
|lun 20/01/2025|1|RelAx4F|Massi,Sciamanna|S4F|7|
|lun 20/01/2025|2|RelAx5E|Massi,Sciamanna|S5E|7|
|mer 22/01/2025|3|RelAx5E|Massi,Sciamanna|S5E|8|
|mer 22/01/2025|4|RelAx4F|Massi,Sciamanna|S4F|8|
|ven 24/01/2025|5|RelAx5LSinf|Massi,Pomili|S5B+S5D|5|
|mer 29/01/2025|3|RelAx5E|Massi,Sciamanna|S5E|9|
|mer 29/01/2025|4|RelAx4F|Massi,Sciamanna|S4F|9|
|mer 29/01/2025|5|CyberSec3B|Sciamanna,Taccari|S3B|1|
|mer 29/01/2025|6|CyberSec4B|Sciamanna,Taccari|S4B|1|
|ven 31/01/2025|4|CyberSec4B|Sciamanna,Taccari|S4B|2|
|ven 31/01/2025|5|LM2|Marrucchiello,Massi|LicMat|1|
|ven 31/01/2025|6|CyberSec3B|Sciamanna,Taccari|S3B|2|
|ven 31/01/2025|6|LM2|Marrucchiello,Massi|LicMat|2|
|lun 03/02/2025|1|RicOp5A|Marinelli,Massi|E5ASIA|1|
|lun 03/02/2025|2|RicOp5A|Marinelli,Massi|E5ASIA|2|
|mar 04/02/2025|2|Crypto4G|Lorenzoni,Taccari|S4G|1|
|mer 05/02/2025|1|Crypto1B|Sciamanna,Taccari|S1B|1|
|mer 05/02/2025|1|Crypto2B|Massi,Pomili|S2B|1|
|mer 05/02/2025|2|RelAx4E|Massi,Pomili|S4E|1|
|mer 05/02/2025|3|RelAx5E|Massi,Sciamanna|S5E|10|
|mer 05/02/2025|4|Crypto2B|Massi,Pomili|S2B|2|
|mer 05/02/2025|5|CyberSec3B|Sciamanna,Taccari|S3B|3|
|ven 07/02/2025|5|LM2|Marrucchiello,Massi|LicMat|3|
|ven 07/02/2025|6|LM2|Marrucchiello,Massi|LicMat|4|
|sab 08/02/2025|4|Crypto1B|Sciamanna,Taccari|S1B|2|
|lun 10/02/2025|1|RicOp5A|Marinelli,Massi|E5ASIA|3|
|lun 10/02/2025|2|RicOp5A|Marinelli,Massi|E5ASIA|4|
|mar 11/02/2025|2|Crypto4G|Lorenzoni,Taccari|S4G|2|
|mer 12/02/2025|1|Crypto1B|Sciamanna,Taccari|S1B|3|
|mer 12/02/2025|2|RelAx4E|Massi,Pomili|S4E|2|
|mer 12/02/2025|4|Crypto2B|Massi,Pomili|S2B|3|
|mer 12/02/2025|5|CyberSec3B|Sciamanna,Taccari|S3B|4|
|ven 14/02/2025|4|CyberSec4B|Sciamanna,Taccari|S4B|3|
|ven 14/02/2025|5|LM2|Marrucchiello,Massi|LicMat|5|
|ven 14/02/2025|6|CyberSec3B|Sciamanna,Taccari|S3B|5|
|ven 14/02/2025|6|LM2|Marrucchiello,Massi|LicMat|6|
|sab 15/02/2025|4|Crypto1B|Sciamanna,Taccari|S1B|4|
|lun 17/02/2025|1|RicOp5A|Marinelli,Massi|E5ASIA|5|
|lun 17/02/2025|2|RicOp5A|Marinelli,Massi|E5ASIA|6|
|mar 18/02/2025|2|Crypto4G|Lorenzoni,Taccari|S4G|3|
|mer 19/02/2025|1|Crypto1B|Sciamanna,Taccari|S1B|5|
|mer 19/02/2025|2|RelAx4E|Massi,Pomili|S4E|3|
|mer 19/02/2025|4|Crypto2B|Massi,Pomili|S2B|4|
|mer 19/02/2025|5|CyberSec3B|Sciamanna,Taccari|S3B|6|
|mer 19/02/2025|6|CyberSec4B|Sciamanna,Taccari|S4B|4|
|sab 22/02/2025|4|Crypto1B|Sciamanna,Taccari|S1B|6|
|lun 24/02/2025|1|RicOp5A|Marinelli,Massi|E5ASIA|7|
|lun 24/02/2025|2|RicOp5A|Marinelli,Massi|E5ASIA|8|
|mer 26/02/2025|1|Crypto1B|Sciamanna,Taccari|S1B|7|
|mer 26/02/2025|2|RelAx4E|Massi,Pomili|S4E|4|
|mer 26/02/2025|4|RelAx4F|Massi,Sciamanna|S4F|10|
|sab 01/03/2025|4|Crypto1B|Sciamanna,Taccari|S1B|8|
|lun 03/03/2025|1|RicOp5A|Marinelli,Massi|E5ASIA|9|
|lun 03/03/2025|2|RicOp5A|Marinelli,Massi|E5ASIA|10|
|mar 04/03/2025|2|Crypto4G|Lorenzoni,Taccari|S4G|4|
|mer 05/03/2025|1|Crypto1B|Sciamanna,Taccari|S1B|9|
|mer 05/03/2025|2|RelAx4E|Massi,Pomili|S4E|5|
|mer 05/03/2025|4|Crypto2B|Massi,Pomili|S2B|5|
|mer 05/03/2025|5|CyberSec3B|Sciamanna,Taccari|S3B|7|
|mer 05/03/2025|6|CyberSec4B|Sciamanna,Taccari|S4B|5|
|ven 07/03/2025|4|CyberSec4B|Sciamanna,Taccari|S4B|6|
|ven 07/03/2025|5|CyberSec4B|Sciamanna,Taccari|S4B|7|
|ven 07/03/2025|5|LM2|Marrucchiello,Massi|LicMat|7|
|ven 07/03/2025|6|CyberSec3B|Sciamanna,Taccari|S3B|8|
|ven 07/03/2025|6|LM2|Marrucchiello,Massi|LicMat|8|
|sab 08/03/2025|4|Crypto1B|Sciamanna,Taccari|S1B|10|
|lun 10/03/2025|2|PhyComp|Massi,Sciamanna|E2B|1|
|mar 11/03/2025|2|Crypto4G|Lorenzoni,Taccari|S4G|5|
|mer 12/03/2025|2|RelAx4E|Massi,Pomili|S4E|6|
|mer 12/03/2025|4|Crypto2B|Massi,Pomili|S2B|6|
|mer 12/03/2025|5|CyberSec3B|Sciamanna,Taccari|S3B|9|
|mer 12/03/2025|6|CyberSec4B|Sciamanna,Taccari|S4B|8|
|lun 17/03/2025|2|PhyComp|Massi,Sciamanna|E2B|2|
|mar 18/03/2025|2|Crypto4G|Lorenzoni,Taccari|S4G|6|
|mer 19/03/2025|2|RelAx4E|Massi,Pomili|S4E|7|
|mer 19/03/2025|4|Crypto2B|Massi,Pomili|S2B|7|
|mer 19/03/2025|5|CyberSec3B|Sciamanna,Taccari|S3B|10|
|mer 19/03/2025|6|CyberSec4B|Sciamanna,Taccari|S4B|9|
|ven 21/03/2025|4|CyberSec4B|Sciamanna,Taccari|S4B|10|
|ven 21/03/2025|5|LM2|Marrucchiello,Massi|LicMat|9|
|ven 21/03/2025|6|CyberSec3B|Sciamanna,Taccari|S3B|11|
|ven 21/03/2025|6|LM2|Marrucchiello,Massi|LicMat|10|
|lun 24/03/2025|2|PhyComp|Massi,Sciamanna|E2B|3|
|mar 25/03/2025|2|Crypto4G|Lorenzoni,Taccari|S4G|7|
|mer 26/03/2025|4|PhyComp|Massi,Sciamanna|E2B|4|
|mer 26/03/2025|5|CyberSec3B|Sciamanna,Taccari|S3B|12|
|mer 26/03/2025|6|CyberSec4B|Sciamanna,Taccari|S4B|11|
|ven 28/03/2025|1|CyberSec3B|Sciamanna,Taccari|S3B|13|
|ven 28/03/2025|3|CyberSec4B|Sciamanna,Taccari|S4B|12|
|ven 28/03/2025|4|CyberSec4B|Sciamanna,Taccari|S4B|13|
|ven 28/03/2025|6|CyberSec3B|Sciamanna,Taccari|S3B|14|
|lun 31/03/2025|2|PhyComp|Massi,Sciamanna|E2B|5|
|mar 01/04/2025|2|Crypto4G|Lorenzoni,Taccari|S4G|8|
|mer 02/04/2025|2|RelAx4E|Massi,Pomili|S4E|8|
|mer 02/04/2025|4|Crypto2B|Massi,Pomili|S2B|8|
|mer 02/04/2025|5|CyberSec3B|Sciamanna,Taccari|S3B|15|
|mer 02/04/2025|6|CyberSec4B|Sciamanna,Taccari|S4B|14|
|ven 04/04/2025|4|CyberSec4B|Sciamanna,Taccari|S4B|15|
|ven 04/04/2025|5|CyberSec3B|Sciamanna,Taccari|S3B|16|
|ven 04/04/2025|5|RelAx5LSinf|Massi,Pomili|S5B+S5D|6|
|ven 04/04/2025|6|CyberSec3B|Sciamanna,Taccari|S3B|17|
|ven 04/04/2025|6|RelAx5LSinf|Massi,Pomili|S5B+S5D|7|
|lun 07/04/2025|2|PhyComp|Massi,Sciamanna|E2B|6|
|mer 09/04/2025|4|PhyComp|Massi,Sciamanna|E2B|7|
|ven 11/04/2025|6|CyberSec3B|Sciamanna,Taccari|S3B|18|
|ven 11/04/2025|6|RelAx5LSinf|Massi,Pomili|S5B+S5D|8|
|mar 15/04/2025|2|Crypto4G|Lorenzoni,Taccari|S4G|9|
|mer 16/04/2025|2|RelAx4E|Massi,Pomili|S4E|9|
|mer 16/04/2025|4|Crypto2B|Massi,Pomili|S2B|9|
|mer 16/04/2025|5|CyberSec3B|Sciamanna,Taccari|S3B|19|
|mer 16/04/2025|6|CyberSec4B|Sciamanna,Taccari|S4B|16|
|mer 23/04/2025|2|RelAx4E|Massi,Pomili|S4E|10|
|mer 23/04/2025|4|Crypto2B|Massi,Pomili|S2B|10|
|mer 23/04/2025|5|CyberSec3B|Sciamanna,Taccari|S3B|20|
|mer 23/04/2025|6|CyberSec4B|Sciamanna,Taccari|S4B|17|
|lun 28/04/2025|2|PhyComp|Massi,Sciamanna|E2B|8|
|mar 29/04/2025|2|Crypto4G|Lorenzoni,Taccari|S4G|10|
|lun 05/05/2025|2|PhyComp|Massi,Sciamanna|E2B|9|
|mer 07/05/2025|4|PhyComp|Massi,Sciamanna|E2B|10|
|mer 07/05/2025|6|CyberSec4B|Sciamanna,Taccari|S4B|18|
|ven 09/05/2025|4|CyberSec4B|Sciamanna,Taccari|S4B|19|
|ven 09/05/2025|5|RelAx5LSinf|Massi,Pomili|S5B+S5D|9|
|ven 09/05/2025|6|RelAx5LSinf|Massi,Pomili|S5B+S5D|10|
|mer 14/05/2025|6|CyberSec4B|Sciamanna,Taccari|S4B|20|





In [8]:
con.close()